# ETL PROJECT
##### Lauren Chavez, James DeCola 

## Part 1 - Importing Requirements

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

#You may need to update config.py in this folder if your connection string and mine are different!
from config import connect_string
print(f'CONNECTION STRING: {connect_string}')

CONNECTION STRING: postgres:Mickey2@@localhost:5432


## Part 2 - Extracting and Transforming 

### Natural Disaster Event counts and Economic Impact

In [2]:
#Load economic damage csv
csv_file = "./Resources/economic-damage-from-natural-disasters.csv"
econ_damage_df = pd.read_csv(csv_file)
econ_damage_df.head()

,Entity,Code,Year,Total economic damage from natural disasters (US$)
0,All natural disasters,NaN,1900,30000000
1,All natural disasters,NaN,1901,0
2,All natural disasters,NaN,1902,0
3,All natural disasters,NaN,1903,480000000
4,All natural disasters,NaN,1904,0


In [3]:
#cleaning up economic damage frame
econ_damage_df.rename(columns={"Entity": "disaster_type", "Year": "year", "Total economic damage from natural disasters (US$)":"econ_damage"}, inplace=True)
econ_damage_df = econ_damage_df[[ "year", "disaster_type", "econ_damage"]]
econ_damage_df.head()

,year,disaster_type,econ_damage
0,1900,All natural disasters,30000000
1,1901,All natural disasters,0
2,1902,All natural disasters,0
3,1903,All natural disasters,480000000
4,1904,All natural disasters,0


In [4]:
#Load disaster count csv
csv_file = "./Resources/number-of-natural-disaster-events.csv"
event_count_df = pd.read_csv(csv_file)
event_count_df.head()

,Entity,Code,Year,Number of reported natural disasters (reported disasters)
0,All natural disasters,NaN,1900,5
1,All natural disasters,NaN,1901,2
2,All natural disasters,NaN,1902,9
3,All natural disasters,NaN,1903,8
4,All natural disasters,NaN,1904,2


In [5]:
#cleaning up disaster count frame
event_count_df.rename(columns={"Entity": "disaster_type", "Year": "year", "Number of reported natural disasters (reported disasters)":"disaster_count"}, inplace=True)
event_count_df = event_count_df[[ "year", "disaster_type", "disaster_count"]]
event_count_df.head()

,year,disaster_type,disaster_count
0,1900,All natural disasters,5
1,1901,All natural disasters,2
2,1902,All natural disasters,9
3,1903,All natural disasters,8
4,1904,All natural disasters,2


In [6]:
#joining frames into one useful frame.
#Not as familiar with pd.merge, but spotchecks showed that ouput 
#was what I'm looking for, and row count is right!

disaster_count_econ = pd.merge(event_count_df, econ_damage_df,  how='left', left_on=['year','disaster_type'], right_on = ['year','disaster_type'])
disaster_count_econ.head()

,year,disaster_type,disaster_count,econ_damage
0,1900,All natural disasters,5,30000000.0
1,1901,All natural disasters,2,0.0
2,1902,All natural disasters,9,0.0
3,1903,All natural disasters,8,480000000.0
4,1904,All natural disasters,2,0.0


In [7]:
#Creating a df that is merged and separated by disaster type - now it starts looking like a relational db table!

disaster_df = disaster_count_econ.pivot_table(['disaster_count','econ_damage'],['year'],'disaster_type')

disaster_df.columns = ['all_disaster_count','drought_count','earthquake_count','extreme_temp_count','extreme_weather_count','flood_count','impact_count','landslide_count','dry_mass_movement_count','volcanic_count','wildfire_count',
                      'all_disaster_cost','drought_cost','earthquake_cost','extreme_temp_cost','extreme_weather_cost','flood_cost','impact_cost','landslide_cost','dry_mass_movement_cost','volcanic_cost','wildfire_cost']

disaster_df= disaster_df.reset_index()
disaster_df = disaster_df.fillna(0)
disaster_df.head(10)

,year,all_disaster_count,drought_count,earthquake_count,extreme_temp_count,extreme_weather_count,flood_count,impact_count,landslide_count,dry_mass_movement_count,...,drought_cost,earthquake_cost,extreme_temp_cost,extreme_weather_cost,flood_cost,impact_cost,landslide_cost,dry_mass_movement_cost,volcanic_cost,wildfire_cost
0,1900,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30000000.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1901,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1902,9.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1903,8.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,480000000.0,0.0,0.0,0.0,0.0,0.0
4,1904,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1905,4.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1906,17.0,1.0,10.0,0.0,3.0,2.0,0.0,0.0,0.0,...,0.0,630750000.0,0.0,20000000.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1907,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,30000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1908,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,116000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1909,11.0,0.0,3.0,0.0,5.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Global Temperatures 

In [8]:
#Load global temps csv file
csv_file = "./Resources/GlobalTemperatures.csv"
yearly_temps_df = pd.read_csv(csv_file)
yearly_temps_df.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#converting dt into a date, removing all data before our scope.
yearly_temps_df['dt'] = pd.to_datetime(yearly_temps_df['dt'])
yearly_temps_df['year'] = pd.DatetimeIndex(yearly_temps_df['dt']).year
yearly_temps_df = yearly_temps_df.loc[yearly_temps_df['dt'] >= '1900']

#removing columns outside of our scope, some renaming
yearly_temps_df.rename(columns={"LandAndOceanAverageTemperature":"avg_temp"}, inplace=True)
yearly_temps_df = yearly_temps_df[['year','avg_temp']]
yearly_temps_df.head()

,year,avg_temp
1800,1900,13.142
1801,1900,13.777
1802,1900,14.400
1803,1900,15.170
1804,1900,15.955


In [10]:
#grouping by year and taking average.  
grouped_temps = yearly_temps_df.groupby(['year'])
yearly_temps_avg = grouped_temps["avg_temp"].mean()
yearly_temps_cleaned = pd.DataFrame({"avg_temp": yearly_temps_avg})

#resetting index and displaying.
yearly_temps_cleaned = yearly_temps_cleaned.reset_index()
yearly_temps_cleaned.head()

,year,avg_temp
0,1900,15.143917
1,1901,15.073333
2,1902,14.958333
3,1903,14.836583
4,1904,14.810417


### People Affected by Natural Disasters 

In [11]:
# Import CSV file into DataFrame
wd_deaths = pd.read_csv('./Resources/wd-deaths-natural-disasters.csv', sep=';')
wd_deaths.head()

,Country Code,Country Name,2014,2013,2012,2011,2010,2009,2008,2007,...,1909,1908,1907,1906,1905,1904,1903,1902,1901,1900
0,AFG,AFGHANISTAN,721.0,155.0,378.0,83.0,350.0,101.0,1334.0,296.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,ALBANIA,NaN,NaN,5.0,NaN,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZA,ALGERIA,NaN,NaN,69.0,10.0,2.0,48.0,93.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASM,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,34.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,ANGOLA,NaN,9.0,0.0,130.0,25.0,178.0,370.0,620.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Drop first two colunns to make a List of Column Names 
year_df = wd_deaths.drop(wd_deaths.columns[[0, 1]], axis=1)

# Create List of Column Names 
column_names = list(year_df.columns.values)

#column_names

In [13]:
# Use Pandas Melt to combine Columns Names into one "Year" Column 
wd_deaths_melt = pd.melt(wd_deaths,
                 id_vars=['Country Code',
                          'Country Name'],
                  value_vars= column_names ,
                  var_name='year')
wd_deaths_melt.head()

,Country Code,Country Name,year,value
0,AFG,AFGHANISTAN,2014,721.0
1,ALB,ALBANIA,2014,NaN
2,DZA,ALGERIA,2014,NaN
3,ASM,AMERICAN SAMOA,2014,NaN
4,AGO,ANGOLA,2014,NaN


In [14]:
# Drop First Two Columns Again 
wd_deaths_melt = wd_deaths_melt.drop(['Country Code', 'Country Name'], axis = 1)

In [15]:
# Groupby Year and use Sum Function to Total year counts 
wd_deaths_melt = wd_deaths_melt.groupby("year").value.sum().reset_index()
wd_deaths_melt = wd_deaths_melt.rename(columns={'value': 'deaths'})

wd_deaths_melt.head()

,year,deaths
0,1900,1267360.0
1,1901,200018.0
2,1902,41475.0
3,1903,6506.0
4,1904,0.0


In [16]:
# Import CSV file into DataFrame
wd_injured = pd.read_csv('./Resources/wd-injured-natural-disasters.csv', sep=';')
wd_injured.head() 

,Country Code,Country Name,2014,2013,2012,2011,2010,2009,2008,2007,...,1909,1908,1907,1906,1905,1904,1903,1902,1901,1900
0,AFG,AFGHANISTAN,0.0,143.0,427.0,115.0,200.0,86.0,182.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,ALBANIA,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZA,ALGERIA,NaN,NaN,101.0,43.0,43.0,48.0,50.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASM,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,ANGOLA,NaN,0.0,0.0,0.0,31.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Drop first two colunns to make a List of Column Names 
year_df = wd_injured.drop(wd_injured.columns[[0, 1]], axis=1)

# Create List of Column Names 
column_names = list(year_df.columns.values)

#column_names

In [18]:
# Use Pandas Melt to combine Columns Names into one "Year" Column 
injured_meltDF = pd.melt(wd_injured,
                 id_vars=['Country Code',
                          'Country Name'],
                  value_vars= column_names ,
                  var_name='year')
injured_meltDF.head()

,Country Code,Country Name,year,value
0,AFG,AFGHANISTAN,2014,0.0
1,ALB,ALBANIA,2014,NaN
2,DZA,ALGERIA,2014,NaN
3,ASM,AMERICAN SAMOA,2014,NaN
4,AGO,ANGOLA,2014,NaN


In [19]:
# Drop First Two Columns Again 
injured_meltDF = injured_meltDF.drop(['Country Code', 'Country Name'], axis = 1)

# Groupby Year and use Sum Function to Total year counts 
injured_meltDF = injured_meltDF.groupby("year").value.sum().reset_index()
injured_meltDF = injured_meltDF.rename(columns={'value': 'injured'})


injured_meltDF.head()


,year,injured
0,1900,0.0
1,1901,0.0
2,1902,0.0
3,1903,23.0
4,1904,0.0


In [20]:
wd_homeless = pd.read_csv('./Resources/wd-homeless-natural-disasters.csv', sep=';')
wd_homeless.head()

,Country Code,Country Name,2014,2013,2012,2011,2010,2009,2008,2007,...,1909,1908,1907,1906,1905,1904,1903,1902,1901,1900
0,AFG,AFGHANISTAN,0.0,0.0,2680.0,9700.0,1000.0,3250.0,180.0,3480.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,ALBANIA,NaN,NaN,0.0,NaN,0.0,150.0,NaN,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZA,ALGERIA,NaN,NaN,0.0,0.0,0.0,2500.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASM,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,ANGOLA,NaN,0.0,0.0,100.0,78875.0,5065.0,0.0,6000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
year_df = wd_homeless.drop(wd_homeless.columns[[0, 1]], axis=1)
column_names = list(year_df.columns.values)

In [22]:
homeless_meltDF = pd.melt(wd_homeless,
                 id_vars=['Country Code',
                          'Country Name'],
                  value_vars= column_names ,
                  var_name='year')
homeless_meltDF.head()

,Country Code,Country Name,year,value
0,AFG,AFGHANISTAN,2014,0.0
1,ALB,ALBANIA,2014,NaN
2,DZA,ALGERIA,2014,NaN
3,ASM,AMERICAN SAMOA,2014,NaN
4,AGO,ANGOLA,2014,NaN


In [23]:
homeless_meltDF = homeless_meltDF.drop(['Country Code', 'Country Name'], axis = 1)
homeless_meltDF = homeless_meltDF.groupby("year").value.sum().reset_index()
homeless_meltDF = homeless_meltDF.rename(columns={'value': 'homeless'})

In [24]:
homeless_meltDF.head()

,year,homeless
0,1900,0.0
1,1901,0.0
2,1902,0.0
3,1903,0.0
4,1904,0.0


In [25]:
# Import CSV file into DataFrame
wd_damage = pd.read_csv('./Resources/wd-total-damage-natural-disasters.csv', sep=';')
wd_damage.head()

,Country Code,Country Name,2014,2013,2012,2011,2010,2009,2008,2007,...,1909,1908,1907,1906,1905,1904,1903,1902,1901,1900
0,AFG,AFGHANISTAN,0.0,0.0,0.0,142000.0,0.0,20000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,ALBANIA,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZA,ALGERIA,NaN,NaN,0.0,779000.0,0.0,0.0,348000.0,43061.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASM,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,ANGOLA,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Drop first two colunns to make a List of Column Names 
year_df = wd_damage.drop(wd_damage.columns[[0, 1]], axis=1)

# Create List of Column Names 
column_names = list(year_df.columns.values)

#column_names

In [27]:
# Use Pandas Melt to combine Columns Names into one "Year" Column 
damage_meltDF = pd.melt(wd_damage,
                 id_vars=['Country Code',
                          'Country Name'],
                  value_vars= column_names ,
                  var_name='year')
damage_meltDF.head()

,Country Code,Country Name,year,value
0,AFG,AFGHANISTAN,2014,0.0
1,ALB,ALBANIA,2014,NaN
2,DZA,ALGERIA,2014,NaN
3,ASM,AMERICAN SAMOA,2014,NaN
4,AGO,ANGOLA,2014,NaN


In [28]:
# Drop First Two Columns Again 
damage_meltDF = damage_meltDF.drop(['Country Code', 'Country Name'], axis = 1)

In [29]:
# Groupby Year and use Sum Function to Total year counts 
damage_meltDF = damage_meltDF.groupby("year").value.sum().reset_index()
damage_meltDF = damage_meltDF.rename(columns={'value': 'damage'})

damage_meltDF.head()

,year,damage
0,1900,30000.0
1,1901,0.0
2,1902,0.0
3,1903,480000.0
4,1904,0.0


In [30]:
# Import CSV file into DataFrame
wd_affected = pd.read_csv('./Resources/wd-total-affected-natural-disasters.csv', sep=';')

wd_affected.head()

,Country Code,Country Name,2014,2013,2012,2011,2010,2009,2008,2007,...,1909,1908,1907,1906,1905,1904,1903,1902,1901,1900
0,AFG,AFGHANISTAN,120000.0,21628.0,51501.0,1762815.0,46200.0,65857.0,452964.0,30255.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,ALBANIA,NaN,NaN,230000.0,NaN,14000.0,6750.0,NaN,225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZA,ALGERIA,NaN,NaN,101.0,793.0,43.0,2548.0,59050.0,1305.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASM,AMERICAN SAMOA,NaN,NaN,NaN,NaN,NaN,2500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,ANGOLA,NaN,1000.0,1833900.0,90684.0,189781.0,251003.0,98837.0,64343.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Drop first two colunns to make a List of Column Names 
year_df = wd_affected.drop(wd_affected.columns[[0, 1]], axis=1)

# Create List of Column Names 
column_names = list(year_df.columns.values)

#column_names

In [32]:
# Use Pandas Melt to combine Columns Names into one "Year" Column 
affected_meltDF = pd.melt(wd_affected,
                 id_vars=['Country Code',
                          'Country Name'],
                  value_vars= column_names ,
                  var_name='year')
affected_meltDF.head()

,Country Code,Country Name,year,value
0,AFG,AFGHANISTAN,2014,120000.0
1,ALB,ALBANIA,2014,NaN
2,DZA,ALGERIA,2014,NaN
3,ASM,AMERICAN SAMOA,2014,NaN
4,AGO,ANGOLA,2014,NaN


In [33]:
# Drop First Two Columns Again 
affected_meltDF = affected_meltDF.drop(['Country Code', 'Country Name'], axis = 1)

# Groupby Year and use Sum Function to Total year counts 
affected_meltDF = affected_meltDF.groupby("year").value.sum().reset_index()
affected_meltDF = affected_meltDF.rename(columns={'value': 'affected'})

affected_meltDF.head()

,year,affected
0,1900,0.0
1,1901,0.0
2,1902,0.0
3,1903,23.0
4,1904,0.0


In [34]:
affected_merged_df = pd.merge(wd_deaths_melt, injured_meltDF , on = "year")
affected_merged_df = pd.merge(affected_merged_df, damage_meltDF , on = "year")
affected_merged_df = pd.merge(affected_merged_df, affected_meltDF , on = "year")
affected_merged_df = pd.merge(affected_merged_df, homeless_meltDF , on = "year")
affected_merged_df.head()

,year,deaths,injured,damage,affected,homeless
0,1900,1267360.0,0.0,30000.0,0.0,0.0
1,1901,200018.0,0.0,0.0,0.0,0.0
2,1902,41475.0,0.0,0.0,0.0,0.0
3,1903,6506.0,23.0,480000.0,23.0,0.0
4,1904,0.0,0.0,0.0,0.0,0.0


In [35]:
#affected_merged_df.set_index('Year', inplace = True)
#affected_merged_df.index.name = "Year"
#affected_merged_df.columns.name = affected_merged_df.index.name
#affected_merged_df.index.name = None

In [36]:
#affected_merged_df.head()

## Emissions per Year 

In [37]:
# Import CSV file into DataFrame
wd_emissions = pd.read_csv('./Resources/wd-fossil-fuel-emissions.csv')
wd_emissions.head()

,Year,Total,Gas,Liquids,Solids,Production,Flaring,Capita
0,1751,3,0,0,3,0,0,NaN
1,1752,3,0,0,3,0,0,NaN
2,1753,3,0,0,3,0,0,NaN
3,1754,3,0,0,3,0,0,NaN
4,1755,3,0,0,3,0,0,NaN


In [38]:
emissions_df = wd_emissions[['Year','Total']]
emissions_df = emissions_df.rename(columns={'Year':'year','Total': 'emissions'})

emissions_df.head()

,year,emissions
0,1751,3
1,1752,3
2,1753,3
3,1754,3
4,1755,3


In [39]:
#To select rows whose column value is in list 
emissions_df = emissions_df[(emissions_df.year >= 1900) & (emissions_df.year <= 2014)]
emissions_df.head()

,year,emissions
149,1900,534
150,1901,552
151,1902,566
152,1903,617
153,1904,624


### Contributing Factors DataFrame

In [40]:
contributing_factors = pd.merge(yearly_temps_cleaned, emissions_df , on = "year")
contributing_factors.head()

,year,avg_temp,emissions
0,1900,15.143917,534
1,1901,15.073333,552
2,1902,14.958333,566
3,1903,14.836583,617
4,1904,14.810417,624


## Part 3 - Loading

In [41]:
#connecting to db
engine = create_engine(f'postgresql://{connect_string}/disaster_db')
engine.table_names()

['disasters', 'affected', 'factors']

In [43]:
#Loading db
disaster_df.to_sql(name='disasters', con=engine, if_exists='append', index=False)

affected_merged_df.to_sql(name='affected', con=engine, if_exists='append', index=False)

contributing_factors.to_sql(name='factors', con=engine, if_exists='append', index=False)

#add other db loads here!

In [44]:
# Confirm Tables were uploaded 
engine.table_names()

['factors', 'disasters', 'affected']

In [45]:
# Verifying Data is in "Disasters" Table
pd.read_sql_query('select * from disasters', con=engine).head()

,year,all_disaster_count,drought_count,earthquake_count,extreme_temp_count,extreme_weather_count,flood_count,impact_count,landslide_count,dry_mass_movement_count,...,drought_cost,earthquake_cost,extreme_temp_cost,extreme_weather_cost,flood_cost,impact_cost,landslide_cost,dry_mass_movement_cost,volcanic_cost,wildfire_cost
0,1900,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30000000.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1901,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1902,9.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1903,8.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,480000000.0,0.0,0.0,0.0,0.0,0.0
4,1904,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# Verifying Data is in "Affected" Table
pd.read_sql_query('select * from affected', con=engine).head()

,year,deaths,injured,damage,affected,homeless
0,1900,1267360.0,0.0,30000.0,0.0,0.0
1,1901,200018.0,0.0,0.0,0.0,0.0
2,1902,41475.0,0.0,0.0,0.0,0.0
3,1903,6506.0,23.0,480000.0,23.0,0.0
4,1904,0.0,0.0,0.0,0.0,0.0


In [47]:
# Verifying Data is in "Temps" Table
pd.read_sql_query('select * from factors', con=engine).head()

,year,avg_temp,emissions
0,1900,15.143917,534
1,1901,15.073333,552
2,1902,14.958333,566
3,1903,14.836583,617
4,1904,14.810417,624


In [48]:
#verifying data is in all tables
pd.read_sql_query('select factors.year, factors.avg_temp, disasters.all_disaster_count from factors join disasters on disasters.year = factors.year where factors.year = 1942', con=engine).head(10)


,year,avg_temp,all_disaster_count
0,1942,15.325333,9.0
1,1942,15.325333,9.0
